In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install transformers

In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2956 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'text', 'instruction', 'output'],
        num_rows: 2956
    })
})

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,instruction,output
0,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
1,,Here is a classification that determines wheth...,"In other words #katandandre, your food was cra...",not_cyberbullying
2,,Here is a classification that determines wheth...,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,,Here is a classification that determines wheth...,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,,Here is a classification that determines wheth...,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)


In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying


In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df

,instruction,output
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,"In other words #katandandre, your food was cra...",not_cyberbullying
2,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
3,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
4,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
...,...,...
2951,U tell ppl its ok if someone ridiculed ur race...,ethnicity
2952,@SexiVixenFromMD you're right the lightskin/da...,ethnicity
2953,"Mad? >>RT @tayyoung_: FUCK OBAMA, dumb ass nigger",ethnicity
2954,@RealKayneWest fuck you you dumb nigger I woul...,ethnicity


In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:
df.head()

,instruction,output
0,"In other words #katandandre, your food was cra...",4
1,"In other words #katandandre, your food was cra...",4
2,Why is #aussietv so white? #MKR #theblock #ImA...,4
3,@XochitlSuckkks a classy whore? Or more red ve...,4
4,"@Jason_Gio meh. :P thanks for the heads up, b...",4


In [ ]:
def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

print(df.head())

                                         instruction  output
0                            words food crapilicious       4
1                            words food crapilicious       4
2                                              white       4
3                   classy whore red velvet cupcakes       4
4  meh p thanks heads concerned another angry dud...       4


In [ ]:
# Function to remove emoticons from text
def remove_emoticons(text):
    return re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)

df['instruction'] = df['instruction'].apply(remove_emoticons)

In [ ]:

#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)


In [ ]:
df['instruction']

0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
# Import libraries
from sklearn.feature_extraction.text import CountVectorizer

# Create the Bag-of-Words model with unigrams
vectorizer = CountVectorizer(ngram_range=(1, 1))
X1 = vectorizer.fit_transform(df["instruction"])

# Print the feature names and the document-term matrix
print("Feature Names:", vectorizer.get_feature_names_out())
print("Document-Term Matrix:\n", X1.toarray())


# Create the Bag-of-Words model with bigrams
vectorizer = CountVectorizer(ngram_range=(2, 2))
X2 = vectorizer.fit_transform(df["instruction"])

# Print the feature names and the document-term matrix
print("Feature Names:", vectorizer.get_feature_names_out())
print("Document-Term Matrix:\n", X2.toarray())

# Create the Bag-of-Words model with trigrams
vectorizer = CountVectorizer(ngram_range=(3, 3))
X3 = vectorizer.fit_transform(df["instruction"])

# Print the feature names and the document-term matrix
print("Feature Names:", vectorizer.get_feature_names_out())
print("Document-Term Matrix:\n", X3.toarray())


Feature Names: ['000' '05' '10' ... 'محمد' 'مدينة' '𝓂𝒶𝓉𝓉𝒽𝑒𝓌']
Document-Term Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Feature Names: ['000 serbian' '05 death' '10 able' ... 'محمد بن' 'مدينة محمد'
 '𝓂𝒶𝓉𝓉𝒽𝑒𝓌 gcu']
Document-Term Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Feature Names: ['000 serbian civilians' '05 death threats' '10 able talk' ...
 'محمد بن زايد' 'مدينة محمد بن' '𝓂𝒶𝓉𝓉𝒽𝑒𝓌 gcu girls']
Document-Term Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
X1.shape #unigram

(2956, 8585)

In [ ]:
X2.shape #bigram

(2956, 29973)

In [ ]:
X3.shape #trigrams

(2956, 30803)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import pandas as pd


In [ ]:
# Concatenate train and validation data for training
X_train = pd.concat([df['instruction']], ignore_index=True)
y_train = pd.concat([df['output']], ignore_index=True)

# Test data
X_test = df['instruction']
y_test = df['output']


In [ ]:
# Create the Bag-of-Words model with unigrams, bigrams, and trigrams
vectorizer = CountVectorizer(ngram_range=(1, 1))
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [ ]:

# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_bow, y_train)
# Make predictions on train set
logreg_train_preds = logreg_model.predict(X_train_bow)
# Evaluate models on train set
logreg_train_scores = (accuracy_score(y_train, logreg_train_preds),
                      precision_score(y_train, logreg_train_preds, average='weighted'),
                      recall_score(y_train, logreg_train_preds, average='weighted'),
                      f1_score(y_train, logreg_train_preds, average='weighted'))

# Print results for train set
print("Train Set Scores:")
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores[0])
print("Precision:", logreg_train_scores[1])
print("Recall:", logreg_train_scores[2])
print("F1-Score:", logreg_train_scores[3])
print()

Train Set Scores:
Logistic Regression Scores:
Accuracy: 0.9824086603518268
Precision: 0.9829907311911265
Recall: 0.9824086603518268
F1-Score: 0.9824649626894011



In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_bow, y_train)
# Make predictions on train set
svm_train_preds = svm_model.predict(X_train_bow)
# Evaluate models on train set
svm_train_scores = (accuracy_score(y_train, svm_train_preds),
                   precision_score(y_train, svm_train_preds, average='weighted'),
                   recall_score(y_train, svm_train_preds, average='weighted'),
                   f1_score(y_train, svm_train_preds, average='weighted'))
# Print results for train set
print("SVM Scores:")
print("Accuracy:", svm_train_scores[0])
print("Precision:", svm_train_scores[1])
print("Recall:", svm_train_scores[2])
print("F1-Score:", svm_train_scores[3])
print()


SVM Scores:
Accuracy: 0.9242219215155616
Precision: 0.9415969694120692
Recall: 0.9242219215155616
F1-Score: 0.9258587328400188



In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X_train_bow, y_train)
# Make predictions on train set
nn_train_preds = nn_model.predict(X_train_bow)
# Evaluate models on train set
nn_train_scores = (accuracy_score(y_train, nn_train_preds),
                  precision_score(y_train, nn_train_preds, average='weighted'),
                  recall_score(y_train, nn_train_preds, average='weighted'),
                  f1_score(y_train, nn_train_preds, average='weighted'))
# Print results for train set
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores[0])
print("Precision:", nn_train_scores[1])
print("Recall:", nn_train_scores[2])
print("F1-Score:", nn_train_scores[3])
print()


Neural Network Scores:
Accuracy: 0.9928958051420839
Precision: 0.992945054655134
Recall: 0.9928958051420839
F1-Score: 0.9929034244144824



In [ ]:
# Tokenization and Padding for LSTM
maxlen = 100  # Maximum sequence length
max_words = 1000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_lstm = tokenizer.texts_to_sequences(X_train)
X_train_lstm = pad_sequences(X_train_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X_train_lstm) > 0.5).astype(int)
# Evaluate models on train set
lstm_train_scores = (accuracy_score(y_train, lstm_train_preds.flatten()),
                    precision_score(y_train, lstm_train_preds.flatten(), average='weighted'),
                    recall_score(y_train, lstm_train_preds.flatten(), average='weighted'),
                    f1_score(y_train, lstm_train_preds.flatten(), average='weighted'))
# Print results for train set
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores[0])
print("Precision:", lstm_train_scores[1])
print("Recall:", lstm_train_scores[2])
print("F1-Score:", lstm_train_scores[3])
print()

Epoch 1/10
93/93 [==============================] - 7s 35ms/step - loss: -10.8265 - accuracy: 0.1461
Epoch 2/10
93/93 [==============================] - 3s 36ms/step - loss: -29.0657 - accuracy: 0.1465
Epoch 3/10
93/93 [==============================] - 4s 48ms/step - loss: -39.1993 - accuracy: 0.1465
Epoch 4/10
93/93 [==============================] - 3s 35ms/step - loss: -48.2095 - accuracy: 0.1465
Epoch 5/10
93/93 [==============================] - 3s 35ms/step - loss: -56.8306 - accuracy: 0.1465
Epoch 6/10
93/93 [==============================] - 4s 42ms/step - loss: -65.2921 - accuracy: 0.1465
Epoch 7/10
93/93 [==============================] - 4s 46ms/step - loss: -73.6259 - accuracy: 0.1465
Epoch 8/10
93/93 [==============================] - 3s 35ms/step - loss: -81.8861 - accuracy: 0.1465
Epoch 9/10
93/93 [==============================] - 3s 36ms/step - loss: -90.0643 - accuracy: 0.1465
Epoch 10/10
93/93 [==============================] - 1s 11ms/step
LSTM Scores:
Accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Make predictions on test set LOGISTIC REGRESSION
logreg_test_preds = logreg_model.predict(X_test_bow)
# Evaluate models on test set
logreg_test_scores = (accuracy_score(y_test, logreg_test_preds),
                      precision_score(y_test, logreg_test_preds, average='weighted'),
                      recall_score(y_test, logreg_test_preds, average='weighted'),
                      f1_score(y_test, logreg_test_preds, average='weighted'))
# Print results for test set
print("Test Set Scores:")
print("Logistic Regression Scores:")
print("Accuracy:", logreg_test_scores[0])
print("Precision:", logreg_test_scores[1])
print("Recall:", logreg_test_scores[2])
print("F1-Score:", logreg_test_scores[3])
print()

Test Set Scores:
Logistic Regression Scores:
Accuracy: 0.9824086603518268
Precision: 0.9829907311911265
Recall: 0.9824086603518268
F1-Score: 0.9824649626894011



In [ ]:
# Make predictions on test set svm
svm_test_preds = svm_model.predict(X_test_bow)
# Evaluate models on test set
svm_test_scores = (accuracy_score(y_test, svm_test_preds),
                   precision_score(y_test, svm_test_preds, average='weighted'),
                   recall_score(y_test, svm_test_preds, average='weighted'),
                   f1_score(y_test, svm_test_preds, average='weighted'))
# Print results for test set
print("SVM Scores:")
print("Accuracy:", svm_test_scores[0])
print("Precision:", svm_test_scores[1])
print("Recall:", svm_test_scores[2])
print("F1-Score:", svm_test_scores[3])
print()

SVM Scores:
Accuracy: 0.9242219215155616
Precision: 0.9415969694120692
Recall: 0.9242219215155616
F1-Score: 0.9258587328400188



In [ ]:
# Make predictions on test set- NEURAL NETWORK

nn_test_preds = nn_model.predict(X_test_bow)
# Evaluate models on test set
nn_test_scores = (accuracy_score(y_test, nn_test_preds),
                  precision_score(y_test, nn_test_preds, average='weighted'),
                  recall_score(y_test, nn_test_preds, average='weighted'),
                  f1_score(y_test, nn_test_preds, average='weighted'))
# Print results for test set
print("Neural Network Scores:")
print("Accuracy:", nn_test_scores[0])
print("Precision:", nn_test_scores[1])
print("Recall:", nn_test_scores[2])
print("F1-Score:", nn_test_scores[3])
print()

Neural Network Scores:
Accuracy: 0.9928958051420839
Precision: 0.992945054655134
Recall: 0.9928958051420839
F1-Score: 0.9929034244144824



In [ ]:
# Make predictions on test set LSTM
X_test_lstm = tokenizer.texts_to_sequences(X_test)
X_test_lstm = pad_sequences(X_test_lstm, maxlen=maxlen)
lstm_test_preds = (lstm_model.predict(X_test_lstm) > 0.5).astype(int)
# Evaluate models on test set
lstm_test_scores = (accuracy_score(y_test, lstm_test_preds.flatten()),
                    precision_score(y_test, lstm_test_preds.flatten(), average='weighted'),
                    recall_score(y_test, lstm_test_preds.flatten(), average='weighted'),
                    f1_score(y_test, lstm_test_preds.flatten(), average='weighted'))
# Print results for test set
print("LSTM Scores:")
print("Accuracy:", lstm_test_scores[0])
print("Precision:", lstm_test_scores[1])
print("Recall:", lstm_test_scores[2])
print("F1-Score:", lstm_test_scores[3])
print()

93/93 [==============================] - 1s 11ms/step
LSTM Scores:
Accuracy: 0.14648173207036536
Precision: 0.021456897830334304
Recall: 0.14648173207036536
F1-Score: 0.03743085865238607



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Create the Bag-of-Words model with bigrams
vectorizer = CountVectorizer(ngram_range=(2, 2))
X2_train_bow = vectorizer.fit_transform(X_train)
X2_test_bow = vectorizer.transform(X_test)


In [ ]:
# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X2_train_bow, y_train)
# Make predictions on train set
logreg_train_preds = logreg_model.predict(X2_train_bow)
# Evaluate models on train set
logreg_train_scores = (accuracy_score(y_train, logreg_train_preds),
                      precision_score(y_train, logreg_train_preds, average='weighted'),
                      recall_score(y_train, logreg_train_preds, average='weighted'),
                      f1_score(y_train, logreg_train_preds, average='weighted'))

# Print results for train set
print("Train Set Scores:")
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores[0])
print("Precision:", logreg_train_scores[1])
print("Recall:", logreg_train_scores[2])
print("F1-Score:", logreg_train_scores[3])
print()

Train Set Scores:
Logistic Regression Scores:
Accuracy: 0.9847767253044655
Precision: 0.9860741634887441
Recall: 0.9847767253044655
F1-Score: 0.9849062651318055



In [ ]:
# Train Logistic Regression model on test dataset
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X2_train_bow, y_train)

# Make Predictions
logreg_test_preds = logreg_model.predict(X2_test_bow)

# Evaluate Performance (Optional)
logreg_test_scores = (accuracy_score(y_test, logreg_test_preds),
                      precision_score(y_test, logreg_test_preds, average='weighted'),
                      recall_score(y_test, logreg_test_preds, average='weighted'),
                      f1_score(y_test, logreg_test_preds, average='weighted'))

# Print results for test set
print("Test Set Scores:")
print("Logistic Regression Scores:")
print("Accuracy:", logreg_test_scores[0])
print("Precision:", logreg_test_scores[1])
print("Recall:", logreg_test_scores[2])
print("F1-Score:", logreg_test_scores[3])


Test Set Scores:
Logistic Regression Scores:
Accuracy: 0.9847767253044655
Precision: 0.9860741634887441
Recall: 0.9847767253044655
F1-Score: 0.9849062651318055


In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X2_train_bow, y_train)
# Make predictions on train set
svm_train_preds = svm_model.predict(X2_train_bow)
# Evaluate models on train set
svm_train_scores = (accuracy_score(y_train, svm_train_preds),
                   precision_score(y_train, svm_train_preds, average='weighted'),
                   recall_score(y_train, svm_train_preds, average='weighted'),
                   f1_score(y_train, svm_train_preds, average='weighted'))
# Print results for train set
print("SVM Scores:")
print("Accuracy:", svm_train_scores[0])
print("Precision:", svm_train_scores[1])
print("Recall:", svm_train_scores[2])
print("F1-Score:", svm_train_scores[3])
print()


SVM Scores:
Accuracy: 0.9035859269282814
Precision: 0.9391207608848351
Recall: 0.9035859269282814
F1-Score: 0.9076884460197912



In [ ]:
print(y_train.dtype)


int64


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train.unique())
y_train_encoded = encoder.transform(y_train)
print(y_train_encoded.dtype)
lstm_model.fit(X_train_lstm, y_train_encoded, epochs=10, batch_size=20)
lstm_train_preds = (lstm_model.predict(X_train_lstm) > 0.5).astype(int)
lstm_train_scores = (accuracy_score(y_train_encoded, lstm_train_preds.flatten()),
                    precision_score(y_train_encoded, lstm_train_preds.flatten(), average='weighted'),
                    recall_score(y_train_encoded, lstm_train_preds.flatten(), average='weighted'),
                    f1_score(y_train_encoded, lstm_train_preds.flatten(), average='weighted'))
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores[0])
print("Precision:", lstm_train_scores[1])
print("Recall:", lstm_train_scores[2])
print("F1-Score:", lstm_train_scores[3])

int64
Epoch 1/10
148/148 [==============================] - 7s 33ms/step - loss: -24.4731 - accuracy: 0.1461
Epoch 2/10
148/148 [==============================] - 6s 41ms/step - loss: -24.4731 - accuracy: 0.1461
Epoch 3/10
148/148 [==============================] - 5s 33ms/step - loss: -24.4731 - accuracy: 0.1461
Epoch 4/10
148/148 [==============================] - 6s 41ms/step - loss: -24.4731 - accuracy: 0.1462
Epoch 5/10
148/148 [==============================] - 5s 33ms/step - loss: -24.4731 - accuracy: 0.1460
Epoch 6/10
148/148 [==============================] - 5s 34ms/step - loss: -24.4731 - accuracy: 0.1464
Epoch 7/10
148/148 [==============================] - 6s 39ms/step - loss: -24.4731 - accuracy: 0.1462
Epoch 8/10
148/148 [==============================] - 5s 33ms/step - loss: -24.4731 - accuracy: 0.1461
Epoch 9/10
148/148 [==============================] - 6s 40ms/step - loss: -24.4731 - accuracy: 0.1462
Epoch 10/10
93/93 [==============================] - 1s 11ms/step
L

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X2_train_bow, y_train)
# Make predictions on train set
nn_train_preds = nn_model.predict(X2_train_bow)
# Evaluate models on train set
nn_train_scores = (accuracy_score(y_train, nn_train_preds),
                  precision_score(y_train, nn_train_preds, average='weighted'),
                  recall_score(y_train, nn_train_preds, average='weighted'),
                  f1_score(y_train, nn_train_preds, average='weighted'))
# Print results for train set
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores[0])
print("Precision:", nn_train_scores[1])
print("Recall:", nn_train_scores[2])
print("F1-Score:", nn_train_scores[3])
print()

Neural Network Scores:
Accuracy: 0.9847767253044655
Precision: 0.9856203891594192
Recall: 0.9847767253044655
F1-Score: 0.9848788677273503



In [ ]:
# Tokenization and Padding for LSTM
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X2_train_lstm = tokenizer.texts_to_sequences(X_train)
X2_train_lstm = pad_sequences(X2_train_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X2_train_lstm, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X2_train_lstm) > 0.5).astype(int)
# Evaluate models on train set
lstm_train_scores = (accuracy_score(y_train, lstm_train_preds.flatten()),
                    precision_score(y_train, lstm_train_preds.flatten(), average='weighted'),
                    recall_score(y_train, lstm_train_preds.flatten(), average='weighted'),
                    f1_score(y_train, lstm_train_preds.flatten(), average='weighted'))
# Print results for train set
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores[0])
print("Precision:", lstm_train_scores[1])
print("Recall:", lstm_train_scores[2])
print("F1-Score:", lstm_train_scores[3])
print()

Epoch 1/10
93/93 [==============================] - 6s 39ms/step - loss: -12.3593 - accuracy: 0.1455
Epoch 2/10
93/93 [==============================] - 4s 43ms/step - loss: -29.6435 - accuracy: 0.1465
Epoch 3/10
93/93 [==============================] - 4s 48ms/step - loss: -39.4396 - accuracy: 0.1465
Epoch 4/10
93/93 [==============================] - 4s 39ms/step - loss: -48.3956 - accuracy: 0.1465
Epoch 5/10
93/93 [==============================] - 4s 39ms/step - loss: -57.0207 - accuracy: 0.1465
Epoch 6/10
93/93 [==============================] - 5s 51ms/step - loss: -65.4601 - accuracy: 0.1465
Epoch 7/10
93/93 [==============================] - 4s 39ms/step - loss: -73.7848 - accuracy: 0.1465
Epoch 8/10
93/93 [==============================] - 4s 39ms/step - loss: -82.0226 - accuracy: 0.1465
Epoch 9/10
93/93 [==============================] - 5s 51ms/step - loss: -90.2097 - accuracy: 0.1465
Epoch 10/10
93/93 [==============================] - 1s 11ms/step
LSTM Scores:
Accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Create the Bag-of-Words model with trigrams
vectorizer = CountVectorizer(ngram_range=(3, 3))
X3_train_bow = vectorizer.fit_transform(X_train)
X3_test_bow = vectorizer.transform(X_test)


In [ ]:
# Train Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X3_train_bow, y_train)
# Make predictions on train set
logreg_train_preds = logreg_model.predict(X3_train_bow)
# Evaluate models on train set
logreg_train_scores = (accuracy_score(y_train, logreg_train_preds),
                      precision_score(y_train, logreg_train_preds, average='weighted'),
                      recall_score(y_train, logreg_train_preds, average='weighted'),
                      f1_score(y_train, logreg_train_preds, average='weighted'))

# Print results for train set
print("Train Set Scores:")
print("Logistic Regression Scores:")
print("Accuracy:", logreg_train_scores[0])
print("Precision:", logreg_train_scores[1])
print("Recall:", logreg_train_scores[2])
print("F1-Score:", logreg_train_scores[3])
print()

Train Set Scores:
Logistic Regression Scores:
Accuracy: 0.9725981055480379
Precision: 0.9765334840065645
Recall: 0.9725981055480379
F1-Score: 0.9729460585545092



In [ ]:
# Make predictions on test set
logreg_test_preds = logreg_model.predict(X3_test_bow)

# Evaluate performance on test set
logreg_test_scores = (accuracy_score(y_test, logreg_test_preds),
                      precision_score(y_test, logreg_test_preds, average='weighted'),
                      recall_score(y_test, logreg_test_preds, average='weighted'),
                      f1_score(y_test, logreg_test_preds, average='weighted'))

# Print results for test set
print("Test Set Scores:")
print("Logistic Regression Scores:")
print("Accuracy:", logreg_test_scores[0])
print("Precision:", logreg_test_scores[1])
print("Recall:", logreg_test_scores[2])
print("F1-Score:", logreg_test_scores[3])

Test Set Scores:
Logistic Regression Scores:
Accuracy: 0.9725981055480379
Precision: 0.9765334840065645
Recall: 0.9725981055480379
F1-Score: 0.9729460585545092


In [ ]:
# Train SVM model
svm_model = SVC()
svm_model.fit(X3_train_bow, y_train)
# Make predictions on train set
svm_train_preds = svm_model.predict(X3_train_bow)
# Evaluate models on train set
svm_train_scores = (accuracy_score(y_train, svm_train_preds),
                   precision_score(y_train, svm_train_preds, average='weighted'),
                   recall_score(y_train, svm_train_preds, average='weighted'),
                   f1_score(y_train, svm_train_preds, average='weighted'))
# Print results for train set
print("SVM Scores:")
print("Accuracy:", svm_train_scores[0])
print("Precision:", svm_train_scores[1])
print("Recall:", svm_train_scores[2])
print("F1-Score:", svm_train_scores[3])
print()

SVM Scores:
Accuracy: 0.8799052774018945
Precision: 0.9285149113084241
Recall: 0.8799052774018945
F1-Score: 0.8873391570899941



In [ ]:
# Train Neural Network model
nn_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
nn_model.fit(X3_train_bow, y_train)
# Make predictions on train set
nn_train_preds = nn_model.predict(X3_train_bow)
# Evaluate models on train set
nn_train_scores = (accuracy_score(y_train, nn_train_preds),
                  precision_score(y_train, nn_train_preds, average='weighted'),
                  recall_score(y_train, nn_train_preds, average='weighted'),
                  f1_score(y_train, nn_train_preds, average='weighted'))
# Print results for train set
print("Neural Network Scores:")
print("Accuracy:", nn_train_scores[0])
print("Precision:", nn_train_scores[1])
print("Recall:", nn_train_scores[2])
print("F1-Score:", nn_train_scores[3])
print()


Neural Network Scores:
Accuracy: 0.9739512855209743
Precision: 0.9772241928310884
Recall: 0.9739512855209743
F1-Score: 0.9741873064442512



In [ ]:
# Tokenization and Padding for LSTM
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X3_train_lstm = tokenizer.texts_to_sequences(X_train)
X3_train_lstm = pad_sequences(X3_train_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X2_train_lstm, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X3_train_lstm) > 0.5).astype(int)
# Evaluate models on train set
lstm_train_scores = (accuracy_score(y_train, lstm_train_preds.flatten()),
                    precision_score(y_train, lstm_train_preds.flatten(), average='weighted'),
                    recall_score(y_train, lstm_train_preds.flatten(), average='weighted'),
                    f1_score(y_train, lstm_train_preds.flatten(), average='weighted'))
# Print results for train set
print("LSTM Scores:")
print("Accuracy:", lstm_train_scores[0])
print("Precision:", lstm_train_scores[1])
print("Recall:", lstm_train_scores[2])
print("F1-Score:", lstm_train_scores[3])
print()

Epoch 1/10
93/93 [==============================] - 7s 55ms/step - loss: -12.5140 - accuracy: 0.1445
Epoch 2/10
93/93 [==============================] - 4s 46ms/step - loss: -30.1270 - accuracy: 0.1465
Epoch 3/10
93/93 [==============================] - 8s 83ms/step - loss: -39.8740 - accuracy: 0.1465
Epoch 4/10
93/93 [==============================] - 4s 40ms/step - loss: -48.7724 - accuracy: 0.1465
Epoch 5/10
93/93 [==============================] - 4s 40ms/step - loss: -57.3382 - accuracy: 0.1465
Epoch 6/10
93/93 [==============================] - 5s 54ms/step - loss: -65.7515 - accuracy: 0.1465
Epoch 7/10
93/93 [==============================] - 4s 41ms/step - loss: -74.0425 - accuracy: 0.1465
Epoch 8/10
93/93 [==============================] - 4s 41ms/step - loss: -82.2674 - accuracy: 0.1465
Epoch 9/10
93/93 [==============================] - 5s 54ms/step - loss: -90.4477 - accuracy: 0.1465
Epoch 10/10
93/93 [==============================] - 1s 11ms/step
LSTM Scores:
Accuracy: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
